In [7]:
# # initiate the graph
# movie_hetero_graph = dgl.heterograph(graph_data) 
# movie_hetero_graph_clean = clean_graph(movie_hetero_graph, users_df, movie_df, ratings_df)

# print(movie_hetero_graph)
# print("-------------------")
# print(movie_hetero_graph_clean)

In [47]:
import dgl
graphs, _ = dgl.load_graphs("/Users/pratikaher/SPRING23/Capstone/GNN_Architecture/run_data/graph_files_subgraph/ecommerce_hetero_graph.dgl")
ecommerce_hetero_graph = graphs[0]

In [48]:
ecommerce_hetero_graph

Graph(num_nodes={'customer': 96516, 'product': 32171},
      num_edges={('customer', 'orders', 'product'): 115609, ('product', 'rev-orders', 'customer'): 115609},
      metagraph=[('customer', 'product', 'orders'), ('product', 'customer', 'rev-orders')])

# Structural Features

### Deep Random Walk

**Network Embeddings**

Network embedding, i.e., network representation learning (NRL), is proposed to **embed network into a low dimensional space while preserving the network structure and property** so that the learned embeddings can be applied to the downstream network tasks. 

* random walk
* deep neural network
* matrix factorization

All these algorithms are proposed for the **homogeneous graphs**.
<br>
<br>


**Deep Random Walk**

Transfer graph into vectors. The representation vector carries structural information about nodes and its neighbor.

Steps:
1. Generate a random walk (a list of nodes walked) for each node
2. To make it a feature (for each node), 
  - Use the classic walk (an 1D array)
  - Make a revision of the walk, e.g. user_only_walk, movie_only_walk (a shorter 1D array)
  - Calculate a score, e.g. cnt of different movies reached (a single value)
  - Transform into embeddings

TODO:
1. Nodes in this graph should have unique ids, otherwise from the embeddings we can't distinguish nodes with same id but different node types.
2. Check that the similarity of embedding reflects the real situation


Backlog:
1. Allow more walks per node (not implemented)


Source:

[DeepWalk: Online Learning of Social Representations](https://arxiv.org/pdf/1403.6652.pdf)

[DeepWalk Implementation](https://towardsdatascience.com/exploring-graph-embeddings-deepwalk-and-node2vec-ee12c4c0d26d)




In [52]:
import random
import numpy as np
from typing import List
import copy
import torch
from torch import nn
from gensim.models import Word2Vec

class DeepWalk:
    def __init__(self, g: dgl.DGLGraph, node_edge_pairs: List, walk_length: int, walks_per_node: int):
        """
        :param walk_length: length of the walk
        :param walks_per_node: number of walks per node
        """
        self.walk_length = walk_length
        self.walk_per_node = walks_per_node # backlog
        self.num_of_nodes = g.number_of_nodes()
        self.num_of_nodes_ntype = []
        for pair in node_edge_pairs:
            ntype = pair[0]
            self.num_of_nodes_ntype.append([ntype, len(g.nodes[ntype])])
        self.walks = self._forward(g, node_edge_pairs)

    def _forward(self, g: dgl.DGLGraph, node_edge_pairs: List) -> dict:
        """
        Generate a random walk for every node in the graph. 
        :param g: Graph
        :param node_edge_pairs: [node type, 1st level edge type, 2nd level edge type]
        :return:
        """
        nodes_walk = {}
        for pair in node_edge_pairs:
            ntype = pair[0]
            nodes_walk[ntype] = []
            for start in g.nodes(ntype).tolist():
                walk = [start]
                for i in range(self.walk_length):
                    current = walk[i]
                    if i%2 == 0:
                        neighbors = g.successors(current, etype=pair[1]).tolist()
                    else:
                        neighbors = g.successors(current, etype=pair[2]).tolist()
                    next = random.choice(neighbors) # random sampling (equal probabilities)
                    walk.append(next) # walk to the next node
                nodes_walk[ntype].append(walk)
        return nodes_walk

    def get_feature_walk(self, node_type_restriction=False):
        """
        Return the walks.
        :param node_type_restriction: Default `False`. If `True`, return walks that contains only the type of the start node.
        :return:
        """
        if not node_type_restriction:
            return self.walks
        else:
            walks = copy.deepcopy(self.walks)
            ntypes = walks.keys()
            for ntype in ntypes:
                for w in walks[ntype]:
                    # Remove nodes with odd indices. In the walk list, even indices have the same node type as start node.
                    del w[1::2]
            return walks

    def get_feature_diversity_score(self):
        """
        Return a score of (cnts of different nodes in the walk)/(walk length).
        """
        walks = self.get_feature_walk(node_type_restriction=True)
        score = {}
        ntypes = walks.keys()
        for ntype in ntypes:
            score[ntype] = []
            for w in walks[ntype]:
                node = w[0]
                distinct_node_cnt = len(set(w))
                if distinct_node_cnt > 0:
                    s = distinct_node_cnt/len(w)
                else:
                    s = 0                
                score[ntype].append([node, s])
        return score

    def get_embedding(self, H):
        """
        Return the embeddings.
        :param H: The output dimension of embeddings. After projection, every node_id becomes an H-dim array.
        :return: Tensors in the shape of : [num of nodes, walk length, H]0
        """
        # create input tensor
        input_walks = []
        for _, k in enumerate(self.walks):
            input_walks.extend([w for w in self.walks[k]])
        input_tensor = torch.tensor(input_walks)

        # train the embedding
        embedding_layer = nn.Embedding(num_embeddings=self.num_of_nodes, embedding_dim=H) # need unique keys for this method
        _embedding = embedding_layer(input_tensor)

        # sort into different ntypes
        embedding_walk = {}
        start = 0
        for pair in self.num_of_nodes_ntype:
            ntype = pair[0]
            n = pair[1]
            embedding_walk[ntype] = _embedding[start:(start + n)]
            start += n
        return embedding_walk
            

In [79]:
### driver ###
deepwalk = DeepWalk(g=ecommerce_hetero_graph, 
                    node_edge_pairs=[['customer', 'orders', 'rev-orders'],['product', 'rev-orders', 'orders']],
                    walk_length=3, 
                    walks_per_node=1)
deep_walk = deepwalk.get_feature_walk()
same_type_walk = deepwalk.get_feature_walk(node_type_restriction=True)
diversity_score = deepwalk.get_feature_diversity_score()
embedding_walk = deepwalk.get_embedding(H=1)

# print(deep_walk)
# print('---------------')
# print(same_type_walk)
# print('---------------')
# print(diversity_score)
# print('---------------')
print(embedding_walk)

{'customer': tensor([[[0.8578],
         [0.8578],
         [0.8578],
         [0.8578]]], grad_fn=<SliceBackward>), 'product': tensor([[[1.1810],
         [0.8578],
         [0.0417],
         [0.8578]]], grad_fn=<SliceBackward>)}


In [80]:
for key in deep_walk:
    deep_walk[key] = torch.tensor(deep_walk[key]).detach()

In [96]:
deep_walk['customer']

tensor([[    0,     0,     0,     0],
        [    1,     0,     6,     0],
        [    2,     0,     6,     0],
        ...,
        [96513, 32168, 96513, 32168],
        [96514, 32169, 96514, 32169],
        [96515, 32170, 96515, 32170]])

In [99]:
embedding_walk

{'customer': tensor([[[0.8578],
          [0.8578],
          [0.8578],
          [0.8578]]], grad_fn=<SliceBackward>),
 'product': tensor([[[1.1810],
          [0.8578],
          [0.0417],
          [0.8578]]], grad_fn=<SliceBackward>)}

### In-degree, Out-degree

In [54]:
def indegree_feature(graph):
    output = {}
    for s_ntype in graph.ntypes:
        for d_ntype in graph.ntypes:
            if s_ntype!=d_ntype:
                print("Calculating indegree for source ntype: ", s_ntype, "and dest ntype: ", d_ntype)
                indegree_sum = None
                for etype in graph.etypes:
                    try:
                        indegree = graph.in_degrees(etype=(s_ntype, etype, d_ntype))
                        if indegree_sum:
                            indegree_sum += indegree
                        else:
                            indegree_sum = indegree
                    except:
                        print(f'no edge type {etype} between source {s_ntype} and dest {d_ntype}')
                        
                indegree_tensor = torch.FloatTensor([[val] for val in indegree_sum])
                output[d_ntype] = indegree_tensor
    return output

def outdegree_feature(graph):
    output = {}
    for s_ntype in graph.ntypes:
        for d_ntype in graph.ntypes:
            if s_ntype!=d_ntype:
                print("Calculating outdegree for source ntype: ", s_ntype, "and dest ntype: ", d_ntype)
                outdegree_sum = None
                for etype in graph.etypes:
                    try:
                        outdegree = graph.out_degrees(etype=(d_ntype, etype, s_ntype))
                        if outdegree_sum:
                            outdegree_sum += outdegree
                        else:
                            outdegree_sum = outdegree
                    except:
                        print(f'no edge type {etype} between source {d_ntype} and dest {s_ntype}')
                        
                outdegree_tensor = torch.FloatTensor([[val] for val in outdegree_sum])
                output[d_ntype] = outdegree_tensor
    return output

In [55]:
### driver ###
indegree = indegree_feature(ecommerce_hetero_graph)
outdegree = outdegree_feature(ecommerce_hetero_graph)

print(indegree, outdegree)


Calculating indegree for source ntype:  customer and dest ntype:  product
no edge type rev-orders between source customer and dest product
Calculating indegree for source ntype:  product and dest ntype:  customer
no edge type orders between source product and dest customer
Calculating outdegree for source ntype:  customer and dest ntype:  product
no edge type orders between source product and dest customer
Calculating outdegree for source ntype:  product and dest ntype:  customer
no edge type rev-orders between source customer and dest product
{'product': tensor([[11.],
        [ 3.],
        [10.],
        ...,
        [ 1.],
        [ 1.],
        [ 1.]]), 'customer': tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])} {'product': tensor([[11.],
        [ 3.],
        [10.],
        ...,
        [ 1.],
        [ 1.],
        [ 1.]]), 'customer': tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1

### PageRank

In [56]:
"""
    PageRank Helper Function
"""
def pagerank_reduce_func(nodes, DAMP=.85):
    msgs = torch.sum(nodes.mailbox['pagerank_pv'], dim=1)
    N = nodes.batch_size()

    pv = (1 - DAMP) / N + DAMP * msgs
    return {'pagerank_pv' : pv}

def pagerank_message_func(edges):
    return {'pagerank_pv' : edges.src['pagerank_pv'] / edges.src['pagerank_deg']}

"""
PageRank

Implements Pagerank features in bypartite GNN

Parameters
----------
g : DGL Heterograph 
    The Graph should contain two node types only.
user_label : string, optional
    Name of the user node
product_label: string, optional
    Name of the product node
edge_label: string, optional
    Name of the user to product edge type
rev_edge_label: string, optional
    Name of the product to user edge type
DAMP: float, optional
    Damp or decay factor. This corresponds to the probability of connections sinking at any giving point (nodes with no outgoing edges). 
    It prevents the sinked nodes from "absorbing" the PageRanks of those pages connected to the sinks. 
reverse: bool, optional
    Whether or not the PageRank algorithm should run on the reverse orientation (products to users)

Returns
    
-------
DGL Heterograph 
    The Graph with pagerank features included in its nodes ("pagerank_pv").
"""
def pagerank(g, user_label = 'user', product_label = 'product', edge_label = 'purchase', rev_edge_label = 'review', DAMP = 0.85, reverse = False):
   
    N = g.number_of_nodes()
    N_user = g.num_src_nodes(user_label)
    N_product = g.num_src_nodes(product_label)
    
    g.nodes[user_label].data['pagerank_pv'] = torch.ones(N_user) / N
    g.nodes[product_label].data['pagerank_pv'] = torch.ones(N_product) / N
    g.nodes[user_label].data['pagerank_deg'] = g.out_degrees(g.nodes(user_label), etype=edge_label).float()
    g.nodes[product_label].data['pagerank_deg'] = g.out_degrees(g.nodes(product_label), etype=rev_edge_label).float()

    g.multi_update_all({edge_label: (pagerank_message_func, pagerank_reduce_func)},"sum")
    
    if(reverse):
        g.multi_update_all({rev_edge_label: (pagerank_message_func, pagerank_reduce_func)},"sum")
 
    dict1 = {}
    dict1[user_label] = torch.unsqueeze(g.nodes[user_label].data['pagerank_pv'], 1) 
    dict1[product_label] = torch.unsqueeze(g.nodes[product_label].data['pagerank_pv'], 1)
    return dict1
    

In [57]:
### driver ###
pagerank = pagerank(ecommerce_hetero_graph, 
             user_label = 'customer', 
             product_label = 'product', 
             edge_label = 'orders', 
             rev_edge_label = 'rev-orders',
             reverse = False)

print(pagerank)
# print(movie_hetero_graph_clean)

{'customer': tensor([[7.7708e-06],
        [7.7708e-06],
        [7.7708e-06],
        ...,
        [7.7708e-06],
        [7.7708e-06],
        [7.7708e-06]]), 'product': tensor([[7.8785e-04],
        [6.8931e-05],
        [6.2649e-04],
        ...,
        [1.5464e-05],
        [1.5464e-05],
        [1.5464e-05]])}


In [58]:
pagerank['product'].shape

torch.Size([32171, 1])

### Concatenate feature tensors and add to graph

In [92]:
def concat_feature_tensors(node_types, **kwargs):
    """
    Take in multiple feature tensors, check if its a valid tensor size, and concatenate them.
    Output: Dict with different ntype as keys, tensors as value.
    """
    out_feature_tensors = {}
    for ntype in node_types:
        for key, value in kwargs.items():
            tensors = value[ntype]        
    
            # sanity check: tensor size
            if tensors.dim() > 3:
                return "Error dimension in feature:{}".format(key)
            if tensors.dim() == 3:
                value[ntype] = tensors.flatten(1, 2)
            # print(key, ntype, value[ntype])
        
        out_feature_tensors[ntype] = torch.cat(tuple([v[ntype] for k, v in kwargs.items()]), 
                                               dim=-1)
    return out_feature_tensors

def add_features_to_graph(g: dgl.DGLGraph, feature_tensor_to_add: torch.tensor):
    """
    Append feature tensors to the nodes in graph.
    """
    # if there's exsiting features in the graph
    exist_features = {}
    for ntype in g.ntypes:
        if ("features" in g.nodes[ntype].data.keys()) and (g.nodes[ntype].data["features"] is not None):
            exist_features[ntype] = g.nodes[ntype].data["features"]
    
    feature_tensor_to_add = concat_feature_tensors(node_types=g.ntypes,
#                                                     exist_feature=exist_features, 
                                                    feature_tensor_to_add=feature_tensor_to_add)
    # append features to the graph
    for ntype in g.ntypes:
        g.nodes[ntype].data["features"] = feature_tensor_to_add[ntype]    
    return g
    

In [93]:
### driver ###
structural_feature_tensors = concat_feature_tensors(node_types=["customer","product"], 
                                          in_degree=indegree,
                                          out_degree=outdegree,
                                          pagerank=pagerank,
                                          walk_embeddings=deep_walk,
                                          )

# movie_hetero_graph_clean = add_features_to_graph(movie_hetero_graph_clean, structural_feature_tensors)

# print(movie_hetero_graph_clean.nodes["user"].data)


In [94]:
ecommerce_hetero_graph_clean = add_features_to_graph(ecommerce_hetero_graph, structural_feature_tensors)

In [95]:
ecommerce_hetero_graph_clean.nodes["customer"].data['features'][0]

tensor([1.0000e+00, 1.0000e+00, 7.7708e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00])

In [87]:
indegree['customer'][0]

tensor([1.])

In [65]:
outdegree['customer'].shape

torch.Size([96516, 1])

In [88]:
pagerank['customer'][0]

tensor([7.7708e-06])

In [91]:
pagerank['customer'][0]

tensor([7.7708e-06])